In [1]:
import tensorflow as tf
import keras_tuner as kt
from pathlib import Path

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.
class_names = ['Koszulka', 'Spodnie', 'Sweter', 'Sukienka', 'Płaszcz', 'Sandał', 'Koszula', 'Tenisówka', 'Torebka', 'Trzewik']

In [3]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=['sgd', 'adam'])
    if optimizer=='sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [4]:
random_search_tuner = kt.RandomSearch(
    build_model, objective="val_accuracy", max_trials=5, overwrite=True, directory='tuner', project_name='fashion_mnist', seed=42
)
random_search_tuner.search(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.7581999897956848

Best val_accuracy So Far: 0.8650000095367432
Total elapsed time: 00h 03m 28s


In [5]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [6]:
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values

{'n_hidden': 8,
 'n_neurons': 37,
 'learning_rate': 0.008547485565344062,
 'optimizer': 'sgd'}

In [7]:
best_trial = random_search_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
n_hidden: 8
n_neurons: 37
learning_rate: 0.008547485565344062
optimizer: sgd
Score: 0.8650000095367432


In [8]:
best_trial.metrics.get_last_value('val_accuracy')

0.8650000095367432

In [9]:
best_model.fit(X_train_full, y_train_full, epochs=10)
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
test_loss

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: nan - accuracy: 0.1005
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 9/10
1875/1875 [==============================] - 4s 2ms/step - loss: nan - accuracy: 0.1000
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: nan - accuracy: 0.1000


nan

In [10]:
test_accuracy

0.10000000149011612

Fit parameters

In [11]:
class CustomClassificationHyperModel(kt.HyperModel):
    def build(self, hp):
        return build_model(hp)
    def fit(self, hp, model, X, y, **kwargs):
        if hp.Boolean('normalize'):
            norm_layer = tf.keras.layers.Normalization()
            X = norm_layer(X)
        return model.fit(X, y, **kwargs)

In [12]:
hyperband_tuner = kt.Hyperband(
    CustomClassificationHyperModel(), objective='val_accuracy', seed=42,
    max_epochs=10, factor=3, hyperband_iterations=2, overwrite=True, 
    directory='tuner', project_name='fashion_mnist_hyperband'
)

In [13]:
root_logdir = Path(hyperband_tuner.project_dir) / "tensorboard"
tensorboard_cb = tf.keras.callbacks.TensorBoard(root_logdir)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2)
hyperband_tuner.search(
    X_train, y_train, epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb, tensorboard_cb]
)

Trial 60 Complete [00h 00m 53s]
val_accuracy: 0.8483999967575073

Best val_accuracy So Far: 0.8855999708175659
Total elapsed time: 00h 25m 24s


In [16]:
best_trial_hyperband = hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial_hyperband.summary()

Trial 0055 summary
Hyperparameters:
n_hidden: 7
n_neurons: 247
learning_rate: 0.0003987720809096887
optimizer: adam
normalize: True
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0049
Score: 0.8855999708175659


In [14]:
bayesian_opt_tuner = kt.BayesianOptimization(
    CustomClassificationHyperModel(), objective='val_accuracy', seed=42,
    max_trials=10, alpha=1e-4, beta=2.6,
    overwrite=True, directory='tuner', project_name='fashion_mnist_bayesian_opt'
)

In [15]:
root_logdir_b = Path(bayesian_opt_tuner.project_dir) / "tensorboard"
tensorboard_cb_b = tf.keras.callbacks.TensorBoard(root_logdir_b)
early_stopping_cb_b = tf.keras.callbacks.EarlyStopping(patience=2)
bayesian_opt_tuner.search(
    X_train, y_train, epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping_cb_b, tensorboard_cb_b]
)

Trial 10 Complete [00h 01m 39s]
val_accuracy: 0.7372000217437744

Best val_accuracy So Far: 0.8845999836921692
Total elapsed time: 00h 14m 31s


In [17]:
best_trial_bayesian = bayesian_opt_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial_bayesian.summary()

Trial 02 summary
Hyperparameters:
n_hidden: 2
n_neurons: 251
learning_rate: 0.001715074355925934
optimizer: adam
normalize: False
Score: 0.8845999836921692
